# LEAGUE OF DATA - DATA PREOPROCESSING

En cualquier proyecto de Machine Learning el preprocesamiento de la información es uno de los pasos más importantes, este nos permite tener un conjunto de datos compacto, completo y limpio, en las mejores condiciones para poder  entrenar un modelo que nos entregue buenos resultados.

En el transcurso de este Notebook se detallará el proceso que se siguió desde la extracción de la información de la base de datos, hasta la generación de los cocientes de cada uno de los equipos.

Los pasos a desarrollar son:
* Descripción de la base de datos.
* Extracción de datos faltantes.
* Enumeración de los partidos.
* Generación de características.
* Construcción de cocientes.

### DESCRIPCIÓN DE LA BASE DE DATOS

La principal fuente de información es la base de datos creada por el equipo de Oracle's Elixir ([disponible aquí](https://oracleselixir.com/)), esta es una fuente de estadísticas de la escena competitiva del videojuego **League of Legends** que cubre los campeonatos de la Liga Norteamericana y Europea (LCS) y la Serie Challenger (CS), LoL Champions Korea (LCK), la Serie Maestra de League of Legends (LMS), la LoL Pro League (LPL), Circuito Brasileiro LoL (CBLoL) y la Liga de Campeones de Turquía (TCL), entre otros.

Este conjuto de datos cuenta con 96 características y 13128 registros, los cuales estan divididos por parejas generando un total de 6564 partidas. Cada una de estas características representa una medida de cada partida, por ejemplo, la duración de la partida, el oro total obtenido, número de objetivos alcanzados y muchos más.

_Es muy importante mencionar que esta no es la base de datos con la cual se van a entrenar el modelo, este conjuntos se utilizará unicamente para caracterizar los equipos._

In [13]:
import numpy as np
import pandas as pd

full_data = pd.read_csv('https://raw.githubusercontent.com/SebasPelaez/LeagueOfData/master/data/OraclesElixir/FullData.csv',low_memory=False) 

full_data.head()

,league,split,date,week,game,patchno,playerid,side,position,player,...,gdat15,xpat10,oppxpat10,xpdat10,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95
0,LCK,2016-2,42578.1131134259,10.3,1,6.14,100,Blue,Team,Team,...,344.0,15359.0,14875.0,484.0,NaN,NaN,NaN,NaN,NaN,NaN
1,LCK,2016-2,42578.1131134259,10.3,1,6.14,200,Red,Team,Team,...,-344.0,14875.0,15359.0,-484.0,NaN,NaN,NaN,NaN,NaN,NaN
2,LCK,2016-2,42578.2347569445,10.3,1,6.14,100,Blue,Team,Team,...,-692.0,16352.0,15554.0,798.0,NaN,NaN,NaN,NaN,NaN,NaN
3,LCK,2016-2,42578.2347569445,10.3,1,6.14,200,Red,Team,Team,...,692.0,15554.0,16352.0,-798.0,NaN,NaN,NaN,NaN,NaN,NaN
4,LCK,2016-2,42587.1137152778,11.5,1,6.14,100,Blue,Team,Team,...,1593.0,17708.0,17378.0,330.0,NaN,NaN,NaN,NaN,NaN,NaN


### EXTRACCIÓN DE DATOS FALTANTES

Los datos de la liga China (LPL) son limitados, ya que ellos no ponen la información estadistica de sus partidas en dominio público, esto genera que dentro del conjunto de datos existan registros sin información en algunas características. 

Debido a que los partidos de los equipos de LPL no entregan información en una cantidad considerable de características, se decide extraer todos aquellos registros que pertenezcan a la liga china.

In [6]:
def ExtractLeague(full_data):
    leagues = full_data[full_data['league']=='LPL']
    extract_data = full_data.drop(leagues.index[:])
    extract_data = extract_data.reset_index(drop=True)
    return extract_data

In [14]:
#Remove LPL League because they don't have full data
extract_data = ExtractLeague(full_data)

### ENUMERACIÓN DE LAS PARTIDAS

Un factor de análisis muy importante dentro de cualquier evento competitivo es el historial de juegos, no es lo mismo un equipo en la semana 1, en la semana 10 y en el final de temporada, además, es muy común en este deporte que cada 6 meses se renueve la plantilla de  cada equipo, lo cual permite que en una nueva temporada el desempeño de los equipos varie notablemente en comparación a la temporada anterior. 

El conjunto de datos nos entrega información de la fecha en la cual se jugó cada partido, esto nos permite llevar continuidad del desempeño de los equipos a traves del tiempo, sin embargo, manipular la información de esta manera es engorroso, lo que nos lleva a plantearnos un método de enumeración de las partidas, el cual agrega una columna más al conjunto de datos, dicha columna tiene por nombre _gamecount_. 

In [10]:
def addGameCount(full_data):
    teams = np.unique(full_data['team'])
    teams = teams.reshape(len(teams),1)
    teams = np.concatenate((teams,np.zeros((len(teams),1))),axis=1)
    gamecount = np.zeros((len(full_data),1))
    for i in full_data.index:
        team = full_data.loc[i]['team']
        team_index = np.where(teams[:,0]==team)
        gamecount[i] = teams[team_index,1]
        teams[team_index,1] = teams[team_index,1] + 1
    
    full_data.insert(loc=0,column='gamecount',value=gamecount) 
    return full_data

In [15]:
#Enumerate the matchs of every team
extract_data = addGameCount(extract_data) 
extract_data.head()

,gamecount,league,split,date,week,game,patchno,playerid,side,position,...,gdat15,xpat10,oppxpat10,xpdat10,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95
0,0.0,LCK,2016-2,42578.1131134259,10.3,1,6.14,100,Blue,Team,...,344.0,15359.0,14875.0,484.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,LCK,2016-2,42578.1131134259,10.3,1,6.14,200,Red,Team,...,-344.0,14875.0,15359.0,-484.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,LCK,2016-2,42578.2347569445,10.3,1,6.14,100,Blue,Team,...,-692.0,16352.0,15554.0,798.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,LCK,2016-2,42578.2347569445,10.3,1,6.14,200,Red,Team,...,692.0,15554.0,16352.0,-798.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,LCK,2016-2,42587.1137152778,11.5,1,6.14,100,Blue,Team,...,1593.0,17708.0,17378.0,330.0,NaN,NaN,NaN,NaN,NaN,NaN


### GENERACIÓN DE CARACTERÍSTICAS

Cómo se menciono anteriormente, el conjunto de datos de Oracle's Elixir no es será el mismo con el cual se entrenará el modelo, sino que apartir de él, se generaran nuevas características que entregan información representativa de cada equipo, y es con esta con información que se construirá el conjunto de datos de entrenamiento.

Lo primero que se debe hacer, es conocer con cuantos equipos cuenta la base de datos. 

In [18]:
full_results = []
teams = np.unique(extract_data['team'])
len(teams)

169

El siguiente paso es definir cuales van a ser las características que van a representar a un equipo.

In [21]:
data_columns = ['matches_played','percentage_blue_win','percentage_red_win',
                'mean_blue_win_time','mean_red_win_time','mean_win_time',
                'mean_kills','mean_deaths','mean_assis','percentage_fb','mean_fb_time',
                'mean_nofb_time','percentage_ft','mean_ft_time','mean_noft_time',
                'mean_total_towers','percentage_fthreetowers','percentage_fdragon','mean_fdragon_time',
                'mean_dragons','percentage_herald','mean_herald_time','mean_barons',
                'mean_fbaron_time','percentage_fbaron','mean_totalgold','mean_golddiff_at10',
                'mean_golddiff_at15','mean_experience_at10','mean_wards','mean_wards_kill','mean_creeps_kill',
                'results']

* **matches_played:** Es la cantidad de partidas en las que ha participado un equipo.
* **percentage_blue_win:** Es el porcentaje de victoria en el lado azul del mapa.
* **percentage_red_win:** Es el porcentaje de victoria en el lado rojo del mapa.
* **mean_blue_win_time:** Es el tiempo medio en el cual un equipo gana estando en el lado azul del mapa.
* **mean_red_win_time:** Es el tiempo medio en el cual un equipo gana estando en el lado rojo del mapa.
* **mean_win_time:** Es el promedio entre *mean_blue_win_time* y *mean_red_win_time*
* **mean_kills:** Es el promedio de asesinatos que el equipo ha realizado.
* **mean_deaths:** Es el promedio de muertes que el equipo ha tenido.
* **mean_assis:** Es el promedio de asistencias a asesinatos que el equipo ha realizado.
* **percentage_fb:** Es el porcentaje de veces que el equipo ha realizado la primera sangre.
* **mean_fb_time:** Es el promedio de tiempo que tarda un equipo en asesinar de primero a un integrante del otro equipo.
* **mean_nofb_time:** Es el promedio de tiempo que tarda el equipo contrario en realizar la primera sangre.
* **percentage_ft:** Es el porcentaje de veces que un equipo fue el primero en destruir una torreta enemiga.
* **mean_ft_time:** Es el promedio de tiempo que tarda un equipo de destruir de primero la torreta enemiga.
* **mean_noft_time:** Es el promedio de tiempo que tarda el equipo contrario en destruir la primer torre.
* **mean_total_towers:** Es el promedio de torretas que destruye un equipo por partida.
* **percentage_fthreetowers:** Es el porcentaje de veces que un equipo es el primero en destruir 3 torretas enemigas.
* **percentage_fdragon:** Es el porcentaje de veces que un equipo asesina de primero a un dragón.
* **mean_fdragon_time:** Es el tiempo promedio que tarda un equipo en asesinar de primero a un dragón.
* **mean_dragons:** Es el promedio de dragones asesinados por un equipo en una partida.
* **percentage_herald:** Es el porcentaje de heraldos de la grieta asesinados.
* **mean_herald_time:** Es el tiempo promedio que tarda un equipo en asesinar a un heraldo de la grieta.
* **mean_barons:** Es el promedio de barones asesinados por un equipo en una partida.
* **mean_fbaron_time:** Es el promedio de tiempo que tarda un equipo en asesinar de primeros al baron.
* **percentage_fbaron:** Es el porcentaje de veces que un equipo asesina de primero a un baron.
* **mean_totalgold:** Es el promedio de oro acumulado por equipo en una partida.
* **mean_golddiff_at10:** Es el promedio de la diferencia de oro que tiene un equipo antes del minuto 10.
* **mean_golddiff_at15:** Es el promedio de la diferencia de oro que tiene un equipo antes del minuto 15.
* **mean_experience_at10:** Es el promedio de experiencia que tiene un equipo antes del minuto 10.
* **mean_wards:** Es el promedio de centinelas puestas por un equipo.
* **mean_wards_kill:** Es el promedio de centinelas asesinadas por un equipo.
* **mean_creeps_kill:** Es el promedio de subditos asesinados por un equipo.
* **results:** Indica si un equipo gano o perdio la partida.

Esto nos da un total de 33 características que describen un equipo. Cabe resaltar que la selección de dichas características se hizo en mayor parte con base a la experiencia que tengo sobre el juego, pero también se tomó en cuanta la información disponible en el conjunto de datos.

In [22]:
def ExtractData(team):
    data = []
    
    #Feature 0: matches_played
    matches_played = len(team)
            
    #Feature 1: percentage_blue_win
    blue_side = team[team['side']=='Blue']
    unique_elements, counts_elements = np.unique(blue_side['result'], return_counts=True)
    if len(unique_elements) < 2:
        if unique_elements == 1:
            wins = counts_elements[0]
            lose = 0
        else:
            wins = 0
            lose = counts_elements[0]
    else:
        wins = counts_elements[1]
        lose = counts_elements[0]
    percentage_blue_win = float("{0:.2f}".format((wins / (wins + lose)) * 100))

    #Feature 2: percentage_red_win
    red_side = team[team['side']=='Red']
    unique_elements, counts_elements = np.unique(red_side['result'], return_counts=True)
    if len(unique_elements) < 2:
        if unique_elements == 1:
            wins = counts_elements[0]
            lose = 0
        else:
            wins = 0
            lose = counts_elements[0]
    else:
        wins = counts_elements[1]
        lose = counts_elements[0]
    percentage_red_win = float("{0:.2f}".format((wins / (wins + lose)) * 100))
    
    #Promedio tiempo de victoria
    side = team[team['side']=='Blue']
    mean_win = side[side['result']==1]
    if(len(mean_win) == 0):
        mean_blue_win_time = 50
    else:
        mean_blue_win_time = np.mean(mean_win['gamelength'])
    
    side = team[team['side']=='Red']
    mean_win = side[side['result']==1]
    if(len(mean_win) == 0):
        mean_red_win_time = 50
    else:
        mean_red_win_time = np.mean(mean_win['gamelength'])    
    
    mean_win_time = (mean_blue_win_time + mean_red_win_time)/2
    
    #Promedio KDA
    mean_kills = np.mean(team['k'])
    mean_deaths = np.mean(team['d'])
    mean_assis = np.mean(team['a'])
       
    #Primeras Sangre
    fb = np.asarray(team['fb'], dtype=int)
    percentage_fb = (np.sum(fb) * 100)/matches_played
        
    fb = team[team['fb']=='1']
    nofb = team[team['fb']=='0']
    mean_fb_time = np.mean(np.asarray(fb['fbtime'], dtype=float)) 
    mean_nofb_time = np.mean(np.asarray(nofb['fbtime'], dtype=float)) 
    
    #TORRES
    percentage_ft = (np.sum(team['ft']) * 100)/matches_played
    tw = team[team['ft']==1]
    notw = team[team['ft']==0]
    mean_ft_time = np.mean(np.asarray(tw['fttime'], dtype=float)) 
    mean_noft_time = np.mean(np.asarray(notw['fttime'], dtype=float))
    
    mean_total_towers = np.mean(team['teamtowerkills'])
    mean_total_towers = np.around(mean_total_towers)
    percentage_fthreetowers = (np.sum(team['firsttothreetowers']) * 100)/matches_played
        
    #DRAGONES
    fd = np.asarray(team['fd'], dtype=int)
    percentage_fdragon = (np.sum(fd) * 100)/matches_played
    
    fd = team[team['fd']==1]
    mean_fdragon_time = np.mean(np.asarray(fb['fdtime'].dropna(how='any'), dtype=float)) 
    mean_dragons = np.mean(team['teamdragkills'])
    mean_dragons = np.around(mean_dragons)
    
    #HERALDOS
    without_nan = team['herald'].dropna(how='any')
    percentage_herald = ((np.sum(without_nan) * 100)/len(without_nan)) if len(without_nan) != 0 else 0
    
    herald = team[team['herald']==1]
    mean_herald_time = np.mean(np.asarray(herald['heraldtime'].dropna(how='any'), dtype=float)) 
    if(percentage_herald == 0):
        mean_herald_time = 20
        
    #BARONES
    fbaron = np.asarray(team['fbaron'], dtype=float)
    percentage_fbaron = (np.sum(fbaron) * 100)/matches_played
    
    fbaron = team[team['fbaron']==1]
    mean_fbaron_time = np.mean(np.asarray(fb['fbarontime'].dropna(how='any'), dtype=float)) 
    mean_barons = np.mean(team['teambaronkills'])
    mean_barons = np.around(mean_barons)
    
    #ORO
    mean_totalgold = np.mean(team['totalgold'] / team['gamelength'])
    mean_golddiff_at10 = np.mean(team['gdat10'])
    mean_golddiff_at15 = np.mean(team['gdat15'])
    
    #EXPERIENCIA
    mean_experience_at10 = np.mean(team['xpdat10'])
    
    #WARDS
    mean_wards = np.mean(team['wards']/ team['gamelength'])
    mean_wards_kill = np.mean(team['wardkills'] / team['gamelength'])
    
    #MINIONS AND NEUTRAL MOSTERS
    mean_creeps_kill = np.mean((team['minionkills']+team['monsterkills'])/ team['gamelength'])
    
    #El resultado siempre lo seteo como -1
    result = -1
    
    data.append(matches_played)
    data.append(percentage_blue_win)
    data.append(percentage_red_win)
    data.append(mean_blue_win_time)
    data.append(mean_red_win_time)
    data.append(mean_win_time)    
    data.append(mean_kills)
    data.append(mean_deaths)
    data.append(mean_assis)    
    data.append(percentage_fb)
    data.append(mean_fb_time)
    data.append(mean_nofb_time)    
    data.append(percentage_ft)
    data.append(mean_ft_time)
    data.append(mean_noft_time)
    data.append(mean_total_towers)
    data.append(percentage_fthreetowers)
    data.append(percentage_fdragon)
    data.append(mean_fdragon_time)
    data.append(mean_dragons)
    data.append(percentage_herald)
    data.append(mean_herald_time)
    data.append(mean_barons)
    data.append(mean_fbaron_time)
    data.append(percentage_fbaron)  
    data.append(mean_totalgold)
    data.append(mean_golddiff_at10)
    data.append(mean_golddiff_at15)
    data.append(mean_experience_at10)
    data.append(mean_wards)
    data.append(mean_wards_kill)
    data.append(mean_creeps_kill)
    data.append(result)
    
    return data

Para generar las características de cada equipo se tomaron en cuenta, unicamente aquellos que tuvieran más de 5 partidas jugadas.

In [23]:
#Generate the team characteristics in every weak, from the fift game.
for j in teams:
    team_data = extract_data[extract_data['team']==j]
    if (len(team_data) >= 5):
        k = 0
        teams_df = pd.DataFrame(columns = data_columns)
        for i in range(5,len(team_data)):
            data = ExtractData(team_data.iloc[0:i,:])
            teams_df.loc[k] = data
            k = k + 1
        full_results.append(teams_df)
    else:
        full_results.append([])

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [30]:
for t in range(0,8):
    print("El equipo:(", teams[t], ") generó", len(full_results[t]), "partidos")

El equipo:( 100 Thieves ) generó 14 partidos
El equipo:( 100 Thieves Academy ) generó 13 partidos
El equipo:( 17 Academy ) generó 4 partidos
El equipo:( AHQ Fighter ) generó 5 partidos
El equipo:( AHQ e-Sports Club ) generó 188 partidos
El equipo:( Afreeca Freecs ) generó 248 partidos
El equipo:( Albus NoX Luna ) generó 17 partidos
El equipo:( Apex ) generó 66 partidos


### CONSTRUCCIÓN DE COCIENTES

Luego de haber caracterizado cada uno de los equipos, el proceso siguiente es encontrar un método que permita comparar dos equipos. Esta comparación debe coincidir con las fechas en las cuales se jugaron los partidos, es por esto que en pasos anteriores se enumeraron las partidas.

El método de comparación entre dos equipos será el cociente (división entre características), dependiendo de cada característica un valor mayor o menor puede tomar diferentes interpretaciones. Por ejemplo, si el cociente de dos equipos en la característica *mean_fb_time* entrega un valor mayor que uno quiere decir que el equipo que juega en el lado Azul tarda mucho tiempo en realizar la primera sangre, lo cual se puede traducir como un mal indicador; mientras que si la característica *mean_nofb_time* entrega un valor mayor que uno quiere decir que al equipo que juega en el lado Azul tardan más tiempo en matar por primera vez durante una partida a uno de sus integrantes, esto se traduce como un buen indicador.